### Analisis de Localidad

In [1]:
import geopandas as gpd
from esda.moran import Moran
from libpysal.weights import Queen
import rasterio
from rasterio.plot import show
from rasterio.features import shapes
from esda.moran import Moran_BV
import imageio.v2 as imageio
import os
from PIL import Image

In [16]:
amenazas = gpd.read_file("../data/processed/amenazas/amenaza_valdivia.shp")

In [ ]:
w = Queen.from_dataframe(amenazas)
moran = Moran(amenazas["gridcode"], w)

/tmp/ipykernel_9587/1739040107.py:1: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(amenazas)
/home/john/Documents/Universidad/Geoinformatica/firescope/.conda/lib/python3.11/site-packages/libpysal/weights/contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
 There is 1 island with id: 58417.
  W.__init__(self, neighbors, ids=ids, **kw)


('WARNING: ', 58417, ' is an island (no neighbors)')


In [ ]:
print(f"I de Moran: {moran.I}, p-value: {moran.p_sim}")

I de Moran: 0.245773610545895, p-value: 0.001


In [33]:
src = rasterio.open("../data/processed/vegetacion_processed.tif")

In [34]:
resultados_formas = shapes(src.read(1), transform=src.transform)

# Crear GeoDataFrame
vegetacion = gpd.GeoDataFrame.from_features(
    [{'geometry': geom, 'properties': {'ndvi': val}} for geom, val in resultados_formas],
    crs=src.crs
)

In [35]:
src.read(1).shape

(1467, 1228)

In [ ]:
w = Queen.from_dataframe(vegetacion)
moran = Moran(vegetacion["ndvi"], w)

/tmp/ipykernel_9587/2546796185.py:1: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(vegetacion)


In [ ]:
print(f"I de Moran: {moran.I}, p-value: {moran.p_sim}")

I de Moran: 0.9077192790247997, p-value: 0.001


In [2]:
src = rasterio.open("../data/processed/topologia_processed.tif")

resultados_formas = shapes(src.read(1), transform=src.transform)

# Crear GeoDataFrame
topologia = gpd.GeoDataFrame.from_features(
    [{'geometry': geom, 'properties': {'altura': val}} for geom, val in resultados_formas],
    crs=src.crs
)

In [ ]:
w = Queen.from_dataframe(topologia)
moran = Moran(topologia["altura"], w)

/tmp/ipykernel_9587/2604982556.py:1: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(topologia)


In [ ]:
print(f"I de Moran: {moran.I}, p-value: {moran.p_sim}")

I de Moran: 0.998001872141377, p-value: 0.001


#### Correlación entre variables

In [19]:
datos = amenazas.sjoin(vegetacion.to_crs(amenazas.crs))

In [ ]:
datos.boundary

In [2]:
src = rasterio.open("../data/processed/topologia_processed.tif")

In [4]:
data = src.read(1)

### Simulación

In [58]:
import moderngl
import numpy as np
import matplotlib.pyplot as plt

tamano_grilla = data.shape

# Crea el contexto y el compute shader
ctx = moderngl.create_context(standalone=True)

compute = ctx.compute_shader(open("../src/shader/simular.glsl").read())

# Estados iniciales
estados = np.zeros(tamano_grilla, dtype=np.int32)
estados[tamano_grilla[0]//2, tamano_grilla[1]//2] = 1

# Genera los buffers para la simulación
buf_a = ctx.buffer(estados.tobytes())
buf_a.bind_to_storage_buffer(0)
buf_b = ctx.buffer(estados.tobytes())
buf_b.bind_to_storage_buffer(1)

def inicializar():
    global buf_a, buf_b
    estados = np.zeros(tamano_grilla, dtype=np.int32)
    estados[tamano_grilla[0]//2, tamano_grilla[1]//2] = 1
    buf_a = ctx.buffer(estados.tobytes())
    buf_a.bind_to_storage_buffer(0)
    buf_b = ctx.buffer(estados.tobytes())
    buf_b.bind_to_storage_buffer(1)

# Ejecuta un paso en la simulación
def paso():
    global buf_a, buf_b
    buf_a.bind_to_storage_buffer(0)
    buf_b.bind_to_storage_buffer(1)
    
    compute.run(group_x=tamano_grilla[0]//32, group_y=tamano_grilla[1]//32)
    ctx.memory_barrier(moderngl.SHADER_STORAGE_BARRIER_BIT)
    buf_a, buf_b = buf_b, buf_a

In [59]:
inicializar()

for i in range(200):
    paso()
    estado_incendio = np.reshape(np.frombuffer(buf_a.read(), dtype=np.int32), tamano_grilla)
    imagen = Image.fromarray(estado_incendio.astype(np.uint8))
    imagen.save(f"../outputs/maps/simulacion/{i}.png")